In [2]:
import geopandas as gp
import csv
import pandas as pd
from shapely.geometry import LineString
import mysql.connector
from shapely.geometry import Polygon, mapping
import datetime
import time
import json 

In [31]:
file = "./KY/KY_final.geojson"    

precincts = gp.read_file(file)
precincts.head()

,id,canonical,type,centerX,centerY,area,neighbors,geometry
0,21179D105,KY Nelson D105,ORIGINAL,37.850026,-85.475196,0.002013050157500042,"21179C101, 21179D101, 21179C102, 21179B104, 21...","MULTIPOLYGON (((-85.47258 37.82863, -85.48071 ..."
1,21149C102,KY McLean C102,ORIGINAL,37.532737,-87.10202099999999,0.005103987640000557,"21183A105, 21149B101, 21059E104, 21059E103, 21...","MULTIPOLYGON (((-87.15110 37.49865, -87.15485 ..."
2,21147F106,KY McCreary F106,ORIGINAL,36.661912,-84.654375,0.02044541949150034,"21147A104, 21147F109, 21147F108, 21147F105, 21...","MULTIPOLYGON (((-84.60974 36.72119, -84.59601 ..."
3,21147A104,KY McCreary A104,ORIGINAL,36.622,-84.499544,0.006988445205999944,"21147F106, 21147F109, 21147B101, 21147B104, 21...","MULTIPOLYGON (((-84.60519 36.59878, -84.60444 ..."
4,21147F109,KY McCreary F109,ORIGINAL,36.655203,-84.517765,0.002377978279499937,"21147F106, 21147A104, 21147A101, 21147A105, 21...","MULTIPOLYGON (((-84.50946 36.62620, -84.51109 ..."


In [22]:
reader = csv.reader(open('./KY_county.csv', 'r'))
d = {}
for row in reader:
    #id = row['County FIPS']
    key = row[0]
    value = row[1]
    d[key] = value
print (d)

{'21001': 'Adair', '21003': 'Allen', '21005': 'Anderson', '21007': 'Ballard', '21009': 'Barren', '21011': 'Bath', '21013': 'Bell', '21015': 'Boone', '21017': 'Bourbon', '21019': 'Boyd', '21021': 'Boyle', '21023': 'Bracken', '21025': 'Breathitt', '21027': 'Breckinridge', '21029': 'Bullitt', '21031': 'Butler', '21033': 'Caldwell', '21035': 'Calloway', '21037': 'Campbell', '21039': 'Carlisle', '21041': 'Carroll', '21043': 'Carter', '21045': 'Casey', '21047': 'Christian', '21049': 'Clark', '21051': 'Clay', '21053': 'Clinton', '21055': 'Crittenden', '21057': 'Cumberland', '21059': 'Daviess', '21061': 'Edmonson', '21063': 'Elliott', '21065': 'Estill', '21067': 'Fayette', '21069': 'Fleming', '21071': 'Floyd', '21073': 'Franklin', '21075': 'Fulton', '21077': 'Gallatin', '21079': 'Garrard', '21081': 'Grant', '21083': 'Graves', '21085': 'Grayson', '21087': 'Green', '21089': 'Greenup', '21091': 'Hancock', '21093': 'Hardin', '21095': 'Harlan', '21097': 'Harrison', '21099': 'Hart', '21101': 'Hender

In [20]:
#Rename columns
#df = df.drop (columns = ['state_coun', 'democrat_p', 'republic_1', 'male_perce', 'republican',
                         #'democrat_p','republic_1','name', 'population', 'democrats', 'SUM',
                       # 'cartodb_id', 'other', 'other_perc', 'male', 'male_perce', 'female',
                        # 'female_per', 'total_regi', 'registered', 'id', 'data', 'dem_rep_di'])

precincts.rename(columns = {"vtd":"id"}, inplace = True)
precincts.rename(columns = {"vtdid":"canonical"}, inplace = True)
#df.rename(columns = {"NEIGHBORS":"neighbors"}, inplace = True)
#df.rename(columns = {"INTPTLON10":"centerY"}, inplace = True)

#Add type columns
precincts.insert(2, 'type', 'ORIGINAL')
precincts.insert(3, 'centerX', None)
precincts.insert(4, 'centerY', None)
precincts.insert(5, 'area', None)
precincts.insert(6, 'neighbors', None)

#df = df.drop ()
precincts.head()

,id,canonical,type,centerX,centerY,area,neighbors,state_coun,geometry
0,21179D105,D105,ORIGINAL,None,None,None,None,21179,"MULTIPOLYGON (((-85.47258 37.82863, -85.48071 ..."
1,21149C102,C102,ORIGINAL,None,None,None,None,21149,"MULTIPOLYGON (((-87.15110 37.49865, -87.15485 ..."
2,21147F106,F106,ORIGINAL,None,None,None,None,21147,"MULTIPOLYGON (((-84.60974 36.72119, -84.59601 ..."
3,21147A104,A104,ORIGINAL,None,None,None,None,21147,"MULTIPOLYGON (((-84.60519 36.59878, -84.60444 ..."
4,21147F109,F109,ORIGINAL,None,None,None,None,21147,"MULTIPOLYGON (((-84.50946 36.62620, -84.51109 ..."


In [23]:
#Canonical name
state = 'KY'
for i, row in precincts.iterrows():   
    id = str(row['canonical'])
    countyId = row['state_coun']
    canonicalId = state + ' ' + d[countyId] + ' ' + id
    precincts.at[i, 'canonical'] = canonicalId

#Drop column

precincts.head()
#df.head()

,id,canonical,type,centerX,centerY,area,neighbors,state_coun,geometry
0,21179D105,KY Nelson D105,ORIGINAL,None,None,None,None,21179,"MULTIPOLYGON (((-85.47258 37.82863, -85.48071 ..."
1,21149C102,KY McLean C102,ORIGINAL,None,None,None,None,21149,"MULTIPOLYGON (((-87.15110 37.49865, -87.15485 ..."
2,21147F106,KY McCreary F106,ORIGINAL,None,None,None,None,21147,"MULTIPOLYGON (((-84.60974 36.72119, -84.59601 ..."
3,21147A104,KY McCreary A104,ORIGINAL,None,None,None,None,21147,"MULTIPOLYGON (((-84.60519 36.59878, -84.60444 ..."
4,21147F109,KY McCreary F109,ORIGINAL,None,None,None,None,21147,"MULTIPOLYGON (((-84.50946 36.62620, -84.51109 ..."


In [ ]:
#Replace multipoly with polygon
arr_polygon = [0] * len(df)
arr_polygon = gp.GeoDataFrame (arr_polygon, columns = ['geometry'])
for index, row in df.iterrows():
    #Progress
    if index % 1000 == 0:
        print (index, datetime.datetime.now())
    polygon = row.geometry[0]
    arr_polygon.loc[index] = polygon
    
df['geometry'] = arr_polygon
df.head()    

In [36]:
#ADD area and centers
for i, row in precincts.iterrows():
    precincts.at[i, 'area'] = row.geometry.area
    bounds = row.geometry.bounds
    centerX = round((bounds[0] + bounds[2]) / 2, 6)
    centerY = round((bounds[1] + bounds[3]) / 2, 6)
    precincts.at[i, 'centerX'] = centerY
    precincts.at[i, 'centerY'] = centerX
    

In [37]:
#precincts = precincts.drop(columns = ['state_coun'])
precincts.head()  

,id,canonical,type,centerX,centerY,area,neighbors,geometry
0,21179D105,KY Nelson D105,ORIGINAL,37.85,-85.4752,0.00201305,"21179C101, 21179D101, 21179C102, 21179B104, 21...","MULTIPOLYGON (((-85.47258 37.82863, -85.48071 ..."
1,21149C102,KY McLean C102,ORIGINAL,37.5327,-87.102,0.00510399,"21183A105, 21149B101, 21059E104, 21059E103, 21...","MULTIPOLYGON (((-87.15110 37.49865, -87.15485 ..."
2,21147F106,KY McCreary F106,ORIGINAL,36.6619,-84.6544,0.0204454,"21147A104, 21147F109, 21147F108, 21147F105, 21...","MULTIPOLYGON (((-84.60974 36.72119, -84.59601 ..."
3,21147A104,KY McCreary A104,ORIGINAL,36.622,-84.4995,0.00698845,"21147F106, 21147F109, 21147B101, 21147B104, 21...","MULTIPOLYGON (((-84.60519 36.59878, -84.60444 ..."
4,21147F109,KY McCreary F109,ORIGINAL,36.6552,-84.5178,0.00237798,"21147F106, 21147A104, 21147A101, 21147A105, 21...","MULTIPOLYGON (((-84.50946 36.62620, -84.51109 ..."


In [ ]:
df.to_file("./KY/KY_precinct_boundary_neighbors_high_precision.geojson", driver='GeoJSON')

In [13]:
### NATIONAL PARKS
file  = './all_national_parks/622305c9-fb01-40e6-97ac-f68aca551c38202034-1-xpq6gt.ugbbb.shp'
national_parks = gp.read_file(file)
national_parks = national_parks.drop(columns = ['OBJECTID', 'UNIT_CODE', 'GIS_Notes', 'DATE_EDIT', 'REGION', 'GNIS_ID', 'CREATED_BY',
                              'METADATA', 'PARKNAME', 'CreationDa', 'Creator', 'EditDate', 'Editor', 
                               'Shape__Are', 'Shape__Len', 'GlobalID'])
national_parks.head()

,UNIT_NAME,STATE,UNIT_TYPE,geometry
0,American Memorial Park,MP,Park,"POLYGON ((16221864.905 1714852.206, 16221867.9..."
1,Cedar Creek and Belle Grove National Historica...,VA,National Historical Park,"MULTIPOLYGON (((-8715759.414 4724026.776, -871..."
2,Little Bighorn Battlefield National Monument,MT,National Monument,"MULTIPOLYGON (((-11953415.314 5703937.476, -11..."
3,Capulin Volcano National Monument,NM,National Monument,"POLYGON ((-11575101.835 4409222.280, -11574603..."
4,Fort Bowie National Historic Site,AZ,National Historic Site,"POLYGON ((-12187153.674 3783174.666, -12187626..."


In [14]:
national_parks_KY = national_parks[national_parks['STATE'] == 'KY']
national_parks_KY.head()

,UNIT_NAME,STATE,UNIT_TYPE,geometry
14,Cumberland Gap National Historical Park,KY,National Historical Park,"POLYGON ((-9326054.844 4376314.695, -9326154.3..."
39,Abraham Lincoln Birthplace National Historical...,KY,National Historical Park,"MULTIPOLYGON (((-9533079.592 4524683.655, -953..."
63,Mammoth Cave National Park,KY,National Park,"POLYGON ((-9592652.158 4462487.456, -9592660.4..."
314,Camp Nelson National Monument,KY,National Monument,"MULTIPOLYGON (((-9419024.033 4549028.652, -941..."


In [15]:
national_parks_KY.to_file("./national_parks/KY/national_parks_KY.geojson", driver='GeoJSON')

In [31]:
### GENERATE NEIGHBORS
start_time = time.time()

for index, row in precincts.iterrows():
    #what complete
    if index % 1000 == 0:
        print (datetime.datetime.now().strftime("%H:%M:%S"), index)
    # get 'not disjoint' countries
    neighbors = precincts[~precincts.geometry.disjoint(row.geometry)].id.tolist()
    # remove own name from the list
    neighbors = [ name for name in neighbors if row.id != name ]
    # add names of neighbors as NEIGHBORS value
    precincts.at[index, 'neighbors'] = ", ".join(neighbors)
    
print ("Finished in ", time.time() - start_time)

20:53:39 0
20:53:50 1000
20:54:00 2000
20:54:10 3000
Finished in  36.50739359855652


In [32]:
precincts.head()

,id,canonical,type,centerX,centerY,area,neighbors,geometry
0,21179D105,KY Nelson D105,ORIGINAL,-85.4752,37.85,0.00201305,"21179C101, 21179D101, 21179C102, 21179B104, 21...","MULTIPOLYGON (((-85.47258 37.82863, -85.48071 ..."
1,21149C102,KY McLean C102,ORIGINAL,-87.102,37.5327,0.00510399,"21183A105, 21149B101, 21059E104, 21059E103, 21...","MULTIPOLYGON (((-87.15110 37.49865, -87.15485 ..."
2,21147F106,KY McCreary F106,ORIGINAL,-84.6544,36.6619,0.0204454,"21147A104, 21147F109, 21147F108, 21147F105, 21...","MULTIPOLYGON (((-84.60974 36.72119, -84.59601 ..."
3,21147A104,KY McCreary A104,ORIGINAL,-84.4995,36.622,0.00698845,"21147F106, 21147F109, 21147B101, 21147B104, 21...","MULTIPOLYGON (((-84.60519 36.59878, -84.60444 ..."
4,21147F109,KY McCreary F109,ORIGINAL,-84.5178,36.6552,0.00237798,"21147F106, 21147A104, 21147A101, 21147A105, 21...","MULTIPOLYGON (((-84.50946 36.62620, -84.51109 ..."


In [38]:
precincts.to_file("./KY/KY_final.geojson", driver='GeoJSON')

In [ ]:
### OVERLAP 

file = "../data/new/KY_precinct_boundary.geojson"
data_temp = gp.read_file(file)

#pltdata_temp=gpd.read_file('data/temp.shp')
#data_temp.head()

In [64]:
def create_neighbors_df(neighbors, df):
    neighbors = neighbors.split(',')
    neighbors_df = df[df['id'].isin(neighbors)][['id', 'geometry']]
    return neighbors_df   

In [88]:
file = "./KY/KY_neighbors.geojson"  
precincts = gp.read_file(file) # open file
precincts = precincts.drop(columns=['area', 'type', 'centerX', 'centerY', 'canonical'])
precincts.head()

,id,neighbors,geometry
0,21179D105,"21179C101, 21179D101, 21179C102, 21179B104, 21...","MULTIPOLYGON (((-85.47258 37.82863, -85.48071 ..."
1,21149C102,"21183A105, 21149B101, 21059E104, 21059E103, 21...","MULTIPOLYGON (((-87.15110 37.49865, -87.15485 ..."
2,21147F106,"21147A104, 21147F109, 21147F108, 21147F105, 21...","MULTIPOLYGON (((-84.60974 36.72119, -84.60625 ..."
3,21147A104,"21147F106, 21147F109, 21147B101, 21147B104, 21...","MULTIPOLYGON (((-84.60519 36.59878, -84.60444 ..."
4,21147F109,"21147F106, 21147A104, 21147A101, 21147A105, 21...","MULTIPOLYGON (((-84.50946 36.62620, -84.50964 ..."


In [97]:
AREA_THRESHOLD = 1e-4
intersections = gp.GeoDataFrame(crs = precincts.crs)

for index, row in precincts.iterrows():
    #For progress
    if index % 1000 == 0:
        print (datetime.datetime.now().strftime("%H:%M:%S"), index)
    neighbors = create_neighbors_df(row['neighbors'], precincts)
    #print(neighbors)
    # check if intersection occured
    overlaps = neighbors[neighbors.geometry.overlaps(row.geometry)]['id'].tolist()
    if len(overlaps) > 0:
        # compare the area with threshold
        for y in overlaps:
            temp_area = gp.overlay(precincts.loc[precincts.id == y,], precincts.loc[precincts.id == row.id], how='intersection')
            temp_area = temp_area.loc[temp_area.geometry.area >= AREA_THRESHOLD]
            
            if temp_area.shape[0] > 0:
                intersections = gp.GeoDataFrame(pd.concat([temp_area, intersections], ignore_index=True),crs = precincts.crs)
                intersections['sorted'] = intersections.apply(lambda y: sorted([y['id_1'],y['id_2']]), axis=1)
                intersections['sorted'] = intersections.sorted.apply(lambda y: ''.join(y))
                intersections = intersections.drop_duplicates('sorted')
                intersections = intersections.reset_index()[['id_1','id_2','geometry']]
                intersections = gp.GeoDataFrame(pd.concat([temp_area, intersections],ignore_index=True),crs=precincts.crs)

16:59:44 0
16:59:57 1000
17:00:11 2000
17:00:23 3000


In [101]:
intersections.shape

(0, 0)

In [57]:
min_area = precincts.at[0, 'area']
max_area = precincts.at[0, 'area']
print(max_area, min_area)
for index, row in precincts.iterrows():
    if min_area > row['area']:
        min_area = row['area']
    if max_area < row['area']:
        max_area = row['area']
        
print(max_area, min_area)

0.0020116508915000234 0.0020116508915000234
0.03215136870249921 7.535204999921032e-07


In [ ]:
### GAP
data_temp_diss = data_temp[['geometry', 'type']]
data_temp_diss = data_temp.dissolve(by='type')
data_temp_diss = df.drop (columns = ['type', 'id', 'canonical', 'centerX', 'centerY', 'area'])

data_temp_diss.head()


In [ ]:
interior = data_temp_diss.interiors.values.tolist()

#print (interior)

gap_list = []
for i in interior:
    if i == []:
        continue
    #print(i[0].coords[:])    
    gap_list.append(Polygon(i[0]))
    
data_gaps = gp.GeoDataFrame(geometry = gap_list,crs=data_temp.crs)
data_gaps['feature_touches'] = data_gaps.geometry.apply(lambda y: data_temp.loc[data_temp.touches(y)]['id'].tolist())
data_gaps.tail()

In [54]:
#ENCLOSED PRECINCTS

add_enclosed_error_querry = ("INSERT INTO mhaczynska.data_error "
               "(ERROR_TYPE, PRECINCT, CENTER_X, CENTER_Y, STATE) "
               "VALUES (%s, %s, %s, %s, %s)")

update_enclosed_error_querry = "UPDATE mhaczynska.data_error SET STATE = %s WHERE PRECINCT = %s"

ERROR_TYPE = 'ENCLOSED'
STATE = 'KY'
enclosed_error_data = []
for index, row in precincts.iterrows():    
    if  len(row.geometry) > 1:
        #print(row.geometry.geoms.coords)
        for i, geom in enumerate(row.geometry):
            if i != 0:
                bounds = geom.bounds
                centerX = round((bounds[0] + bounds[2]) / 2, 6)
                centerY = round((bounds[1] + bounds[3]) / 2, 6)
                #enclosed_row = (ERROR_TYPE, row['id'], centerX, centerY, STATE)
                enclosed_row = (STATE, row['id'])
                enclosed_error_data.append(enclosed_row)
                
cursor.executemany(update_enclosed_error_querry, enclosed_error_data)
mydb.commit()
#print(enclosed_error_data)
   

In [18]:
### ANOMALOUS DATA
file = "./demographic_KY/precinct_demographic_KY.geojson"
demographics = gp.read_file(file)

file = "./election/KY_president2016.json"
president = pd.read_json(file)  

file = "./election/KY_congress2016.json"
congress2016 = pd.read_json(file)

In [8]:
#demographics.drop_duplicates(subset ="id", 
                   #  keep = 'first', inplace = True)
#demographics.to_file("./demographic_KY/precinct_demographic_KY.geojson", driver='GeoJSON')

In [19]:
print(president.shape)
print(congress2016.shape)
print(demographics.shape)
congress2016.head()

(3583, 3)
(3583, 3)
(3583, 9)


,id,democrat,republican
0,21179D105,448,611
1,21149C102,186,391
2,21147F106,15,83
3,21147A104,46,250
4,21147F109,14,71


In [30]:
anomalous_data = []
for index, row in demographics.iterrows():    
    anomalous = False
    
    if row['total'] <= 10:
        anomalous = True       
        
    if president.at[index, 'democrat'] == 0 and president.at[index, 'republican'] == 0:
        anomalous = True
    
    if congress2016.at[index, 'democrat'] == 0 and congress2016.at[index, 'republican'] == 0:        
        anomalous = True
        
    if president.at[index, 'democrat'] + president.at[index, 'republican'] > row['total']:
        anomalous = True
        
    if anomalous:
        anomalous_data.append(row['id'])

print(len(anomalous_data))

102


In [33]:
add_error_querry = ("INSERT INTO mhaczynska.data_error "
               "(ERROR_TYPE, PRECINCT, CENTER_X, CENTER_Y, STATE) "
               "VALUES (%s, %s, %s, %s, %s)")

STATE = 'KY'
ERROR_ANOMALOUS_TYPE = 'ANOMALOUS'
anomalous = []

for precinct_id in anomalous_data:
    centerX = precincts[precincts['id'] == precinct_id]['centerX'].values[0]
    centerY = precincts[precincts['id'] == precinct_id]['centerY'].values[0]
    anomalous_row = (ERROR_ANOMALOUS_TYPE, precinct_id, centerX, centerY, STATE)
    anomalous.append(anomalous_row)

cursor.executemany(add_error_querry, anomalous)
mydb.commit()    
print(anomalous)

[('ANOMALOUS', '21107A202', '37.170822', '-87.36978499999999', 'KY'), ('ANOMALOUS', '21183C105', '37.380751', '-87.04589199999999', 'KY'), ('ANOMALOUS', '21101D106', '37.830354', '-87.578553', 'KY'), ('ANOMALOUS', '21073B109', '38.20205', '-84.83975100000001', 'KY'), ('ANOMALOUS', '21115C109', '37.897114', '-82.805302', 'KY'), ('ANOMALOUS', '21027C101', '37.943862', '-86.46383299999999', 'KY'), ('ANOMALOUS', '21099C101', '37.21442', '-85.7341', 'KY'), ('ANOMALOUS', '21093G104', '37.549678', '-86.038415', 'KY'), ('ANOMALOUS', '21101E111', '37.862333', '-87.570063', 'KY'), ('ANOMALOUS', '21107A205', '37.16727', '-87.52719399999999', 'KY'), ('ANOMALOUS', '21111G149', '38.216582', '-85.669966', 'KY'), ('ANOMALOUS', '21111H150', '38.195672', '-85.704984', 'KY'), ('ANOMALOUS', '21111D136', '38.19028', '-85.60960300000001', 'KY'), ('ANOMALOUS', '21029X001', '37.92058', '-85.836782', 'KY'), ('ANOMALOUS', '21197D103', '37.79445', '-83.69995', 'KY'), ('ANOMALOUS', '21093G108', '37.7182', '-86.11

In [3]:
#POPULATE DATABASE
mydb = mysql.connector.connect(
    host="mysql3.cs.stonybrook.edu",
    user='mhaczynska',
    passwd="110670160",
    database = 'mhaczynska'    
)

cursor = mydb.cursor()
print (mydb)

In [4]:
file = "./KY/KY_final.geojson" 
precincts = gp.read_file(file)
precincts.head()

,id,canonical,type,centerX,centerY,area,neighbors,geometry
0,21179D105,KY Nelson D105,ORIGINAL,37.850026,-85.475196,0.002013050157500042,"21179C101, 21179D101, 21179C102, 21179B104, 21...","MULTIPOLYGON (((-85.47258 37.82863, -85.48071 ..."
1,21149C102,KY McLean C102,ORIGINAL,37.532737,-87.10202099999999,0.005103987640000557,"21183A105, 21149B101, 21059E104, 21059E103, 21...","MULTIPOLYGON (((-87.15110 37.49865, -87.15485 ..."
2,21147F106,KY McCreary F106,ORIGINAL,36.661912,-84.654375,0.02044541949150034,"21147A104, 21147F109, 21147F108, 21147F105, 21...","MULTIPOLYGON (((-84.60974 36.72119, -84.59601 ..."
3,21147A104,KY McCreary A104,ORIGINAL,36.622,-84.499544,0.006988445205999944,"21147F106, 21147F109, 21147B101, 21147B104, 21...","MULTIPOLYGON (((-84.60519 36.59878, -84.60444 ..."
4,21147F109,KY McCreary F109,ORIGINAL,36.655203,-84.517765,0.002377978279499937,"21147F106, 21147A104, 21147A101, 21147A105, 21...","MULTIPOLYGON (((-84.50946 36.62620, -84.51109 ..."


In [11]:
add_precinct_querry = ("INSERT INTO mhaczynska.precinct "
               "(GEO_ID, NAME, STATE_NAME, CENTER_X, CENTER_Y, NEIGHBORS, PRECINCT_GEOJSON) "
               "VALUES (%s, %s, %s, %s, %s, %s, %s)")

update_precinct_querry = ("UPDATE mhaczynska.precinct SET PRECINCT_GEOJSON = %s WHERE GEO_ID = %s")

state = 'KY'
add_precinct = []
for index, row in precincts.iterrows():
    geometry_str = row.geometry.__geo_interface__
    geometry_str = json.dumps(geometry_str)
    #precinct_data = (row['id'], row['canonical'], state, row['centerX'], row['centerY'], row['neighbors'], geometry_str)
    update_precinct = (geometry_str, row['id'])
    #add_precinct.append(update_precinct)
    #cursor.execute(update_precinct_querry, update_precinct)
   #mydb.commit()
    print(update_precinct)
    
    
#cursor.executemany(update_precinct_querry, add_precinct)
#mydb.commit()

('{"type": "MultiPolygon", "coordinates": [[[[-85.47258, 37.82863], [-85.480705, 37.833658], [-85.483432, 37.832076], [-85.485055, 37.829769], [-85.486626, 37.829046], [-85.490692, 37.828234], [-85.501467, 37.823919], [-85.505423, 37.824331], [-85.509373, 37.827103], [-85.510949, 37.827476], [-85.511944, 37.827021], [-85.51379, 37.829486], [-85.513464, 37.830084], [-85.514356, 37.83287], [-85.51407, 37.835353], [-85.516057, 37.837463], [-85.515488, 37.838605], [-85.515911, 37.839364], [-85.507489, 37.846942], [-85.488495, 37.845458], [-85.490316, 37.846846], [-85.483986, 37.851838], [-85.484739, 37.852308], [-85.484193, 37.853356], [-85.483071, 37.85351], [-85.484485, 37.854416], [-85.485705, 37.85602], [-85.484361, 37.857102], [-85.484715, 37.857641], [-85.484425, 37.858987], [-85.485431, 37.861046], [-85.485238, 37.862854], [-85.486355, 37.865171], [-85.485878, 37.86694], [-85.487008, 37.868115], [-85.487958, 37.870186], [-85.457893, 37.875105], [-85.434335, 37.876133], [-85.435696, 

('{"type": "MultiPolygon", "coordinates": [[[[-84.837709, 38.212632], [-84.84144, 38.210217], [-84.846405, 38.208863], [-84.850349, 38.210334], [-84.849293, 38.216009], [-84.850745, 38.215837], [-84.852678, 38.216159], [-84.850042, 38.215975], [-84.84626, 38.217399], [-84.844005, 38.217357], [-84.83925, 38.214331], [-84.837709, 38.212632]]]]}', '21073C102')
('{"type": "MultiPolygon", "coordinates": [[[[-86.392626, 36.753474], [-86.404475, 36.752398], [-86.409173, 36.752637], [-86.405769, 36.776187], [-86.331028, 36.827084], [-86.32925, 36.822836], [-86.329899, 36.82027], [-86.335623, 36.820616], [-86.337289, 36.820153], [-86.340559, 36.820191], [-86.34039, 36.81801], [-86.341657, 36.816609], [-86.34172, 36.81588], [-86.340177, 36.812763], [-86.339356, 36.812436], [-86.33451, 36.812037], [-86.333069, 36.81372], [-86.330157, 36.813944], [-86.327569, 36.813379], [-86.326502, 36.812556], [-86.326423, 36.81197], [-86.3273, 36.811047], [-86.331854, 36.81021], [-86.33482, 36.808898], [-86.337

('{"type": "MultiPolygon", "coordinates": [[[[-83.706525, 38.356504], [-83.700365, 38.376223], [-83.702025, 38.378726], [-83.700933, 38.378729], [-83.691569, 38.392407], [-83.692162, 38.393327], [-83.690526, 38.392528], [-83.684887, 38.39718], [-83.675914, 38.399123], [-83.674888, 38.400718], [-83.675541, 38.402599], [-83.675382, 38.403959], [-83.654711, 38.402997], [-83.654802, 38.402126], [-83.651831, 38.39911], [-83.652287, 38.392394], [-83.64456, 38.391672], [-83.64377, 38.390252], [-83.632611, 38.390672], [-83.631434, 38.388926], [-83.629, 38.387535], [-83.628695, 38.386152], [-83.62684, 38.385728], [-83.622073, 38.381854], [-83.618641, 38.381329], [-83.617104, 38.380071], [-83.616334, 38.380373], [-83.615245, 38.379547], [-83.616149, 38.37887], [-83.616898, 38.377005], [-83.616339, 38.376183], [-83.612188, 38.373514], [-83.612897, 38.371741], [-83.610783, 38.370807], [-83.607757, 38.37113], [-83.608954, 38.369227], [-83.608796, 38.367412], [-83.612499, 38.365852], [-83.614668, 38

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [19]:
precincts.shape

(3583, 8)

In [14]:
#ADD NEIGHBORS
add_neighbor_querry = ("INSERT INTO mhaczynska.neighbors "
               "(PRECINCT, NEIGHBOR_GEOID) "
               "VALUES (%s, %s)")

neighbor_data = []
for index, row in precincts.iterrows():
    neighbors = row['neighbors']
    neighbors = neighbors.split(",")
    for neighbor in neighbors:
        neighbor_row = (row['id'], neighbor.strip())
        neighbor_data.append(neighbor_row)        
    
cursor.executemany(add_neighbor_querry, neighbor_data)
mydb.commit()

In [16]:
## OVERLAP 
add_error_querry = ("INSERT INTO mhaczynska.data_error "
               "(ERROR_TYPE, PRECINCT, CENTER_X, CENTER_Y, STATE) "
               "VALUES (%s, %s, %s, %s, %s)")

file = './KY/ky_points_intersection.geojson'
intersections = gp.read_file(file)
intersections_data = []
ERROR_INTERSECTIONS_TYPE = 'OVERLAP'
STATE = 'KY'
for index, row in intersections.iterrows():
    point = list(row.geometry.coords)
    intersection_row = (ERROR_INTERSECTIONS_TYPE, row['vtd'], point[0][0], point[0][1], STATE)
    intersections_data.append(intersection_row)

cursor.executemany(add_error_querry, intersections_data)
mydb.commit()    
#print(intersections_data)

In [17]:
cursor.close()
mydb.close()

In [13]:
precincts[precincts['id'] == '21111J131'].geometry.head()

9    MULTIPOLYGON (((-85.80563 38.17273, -85.81668 ...
Name: geometry, dtype: geometry